In [1]:
pip install notebook


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install vtk


Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install ipywidgets

In [7]:
#importing the necessary libraries
import numpy as np
import plotly.graph_objects as go
from vtk import *
import plotly.express as px
from plotly.subplots import make_subplots
import collections
import ipywidgets as widgets
from IPython.display import display

In [8]:

# Parse the grid data
reader = vtkXMLImageDataReader()
# Change the location of the file as per your location
reader.SetFileName('C:/Users/Yashwanth Holla/mixture.vti')
reader.Update()
data = reader.GetOutput()
dim = data.GetDimensions()

coordinates = []
for i in range(dim[0]):
    for j in range(dim[1]):
        for k in range(dim[2]):
            point_id = i * dim[1] * dim[2] + j * dim[2] + k
            coordinate = data.GetPoint(point_id)
            coordinates.append(coordinate)

X = [coord[0] for coord in coordinates]
Y = [coord[1] for coord in coordinates]
Z = [coord[2] for coord in coordinates]

imagefile_array = data.GetPointData().GetArray('ImageFile')

values = []
for i in range(dim[0]):
    for j in range(dim[1]):
        for k in range(dim[2]):
            point_id = i * dim[1] * dim[2] + j * dim[2] + k
            value = imagefile_array.GetTuple1(point_id)
            values.append(value)





In [9]:
#Creating the isosurface
isosurface = go.Isosurface(
    x=X,
    y=Y,
    z=Z,
    value=values,
    isomin=0.0,
    isomax=0.0,
    cauto = False,
    cmin = min(values),
    cmax = max(values),
    opacity=1,  
    caps=dict(x_show=False, y_show=False,z_show=False),
    colorscale='plasma',
    showscale=False
)
fig = go.FigureWidget(data=[isosurface])

# Creating a histogram trace
fig12 = px.histogram(values, nbins=30)

# Converting the Plotly Express figure to a FigureWidget
fig2 = go.FigureWidget(fig12)

#Implementing the slider widget
slider = widgets.FloatSlider(
    value=0.0,
    min=min(values),
    max=max(values),
    step=0.01,
    description='Isoval:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
# Update layout of fig to set the figure size
fig.update_layout(
    scene=dict(
        xaxis=dict(title='X', showticklabels=False),
        yaxis=dict(title='Y', showticklabels=False),
        zaxis=dict(title='Z', showticklabels=False)
    ),
   
)

# Update layout of fig2 to set the figure size to match fig
fig2.update_layout(
    xaxis=dict(title='Vortex scalar values'),
    yaxis=dict(title='Frequency'),
    showlegend=False
   
)


# Creating the Reset Button
button = widgets.Button(
    description='Reset',
    disabled=False,
    button_style='', 

)
#Defining the function for the reset button
def on_button_click(button):
   
    slider.value = 0.0
    isovalue = 0.0
    fig.data[0].isomin=isovalue
    fig.data[0].isomax=isovalue
    fig2.data[0].x = values
    
#updating the isosurface plot
def update_isosurface(change):
    
    isovalue = round(change.new, 2)
    isomin = isovalue
    isomax = isovalue
    fig.data[0].isomin = isomin
    fig.data[0].isomax = isomax
    filtered_values = [val for val in values if val >= change.new - 0.25 and val <= change.new + 0.25]
    fig2.data[0].x = filtered_values


slider.observe(update_isosurface, names='value')

button.on_click(on_button_click)

total_display = widgets.VBox([
    widgets.HBox([slider, button]), 
    widgets.HBox([fig,fig2])
])
#displaying the plots
display(total_display)
